In [16]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()

from sklearn.svm import SVC
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers
import scipy.io
import time


## Load data

In [8]:
mat = scipy.io.loadmat('linear_svm.mat')
X_train = mat["X_train"]
X_test = mat["X_test"]
y_train = mat["labels_train"]
y_test = mat["labels_test"]

## Define SVM problem and solve it with CVXOPT

In [ ]:
C = 5
X_dash = y_train * X_train
H = np.dot(X_dash , X_dash.T)
N,M = X_train.shape
y_train = y_train.reshape(-1,1) * 1.

cvxopt_solvers.options['reltol'] = 1e-5

P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((N, 1)))
G = cvxopt_matrix(np.vstack((np.eye(N)*-1,np.eye(N))))
h = cvxopt_matrix(np.hstack((np.zeros(N), np.ones(N) * C)))
A = cvxopt_matrix(y_train.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
t1 = time.time()
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
t2 = time.time()
lambdas = np.array(sol['x'])

In [29]:
# Extract parameters w and b
w = np.matmul((y_train * lambdas).T, X_train).reshape(-1,1)
S = ((lambdas > lambdas.mean()) * (lambdas < C)).flatten()
b = (y_train[S] - np.dot(X_train[S], w))[0]


In [ ]:
# Train a linear SVM using sklearn for comparison

clf = SVC(C = 5, kernel = 'linear')
clf.fit(X_train, y_train.ravel()) 

(clf.predict(X_test) == y_test.T).sum() / len(y_test)

## Evaluate model
On test set

In [ ]:
print("************ Accuracy *****************")
(np.sign(np.matmul(X_test, w) + b) == y_test).sum() / y_test.shape[0]

In [ ]:
print(f"Time for execution: {t2-t1}") 

In [13]:
# Save parameters
np.save("w_OtSD", w)
np.save("b_OtSD", b)